In [1]:
from deepsight.zoo.foodwaste import create_classification_dataset,get_label_mapping,translations_de_en
from deepsight.core.data import ClassificationVisionDataLoader
from deepsight.integrations import DeepchecksRunner
from deepsight.utils import DeepchecksConfig
from deepsight.zoo.timm_models import CLIPModel

import json

/home/fadelco/workspace/deepsight/.venv/lib/python3.11/site-packages/deepchecks/core/serialization/dataframe/html.py:16: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.



In [2]:
ing2name, ing2label = get_label_mapping()

Getting label mapping: 100%|██████████| 375/375 [00:04<00:00, 92.63it/s] 


In [3]:
ing2name

OrderedDict([(90169064, 'dressing_portion'),
             (90284505, 'sauerkraut'),
             (90764221, 'eisbergsalat'),
             (90776750, 'paprika'),
             (90799084, 'reibekuchen'),
             (91366696, 'rotkohl'),
             (91369326, 'zucchini'),
             (91388363, 'pilze'),
             (91412043, 'linsen'),
             (91428454, 'apfelmus'),
             (91428764, 'sahne'),
             (91434330, 'grüne_bohnen'),
             (91434357, 'blumenkohl'),
             (91511750, 'erbsen'),
             (91511769, 'möhre'),
             (91511777, 'reis'),
             (91511785, 'kartoffelwürfel'),
             (91511793, 'kartoffeln'),
             (91511823, 'braune_sauce'),
             (91511831, 'helle_sauce'),
             (91533770, 'wirsing'),
             (91533800, 'hähnchen'),
             (91539999, 'hähnchenstreifen'),
             (91551949, 'bratenjus'),
             (91555332, 'zwiebel'),
             (91560638, 'semmelknödel'),
       

In [4]:
ing2label

OrderedDict([(90169064, 0),
             (90284505, 1),
             (90764221, 2),
             (90776750, 3),
             (90799084, 4),
             (91366696, 5),
             (91369326, 6),
             (91388363, 7),
             (91412043, 8),
             (91428454, 9),
             (91428764, 10),
             (91434330, 11),
             (91434357, 12),
             (91511750, 13),
             (91511769, 14),
             (91511777, 15),
             (91511785, 16),
             (91511793, 17),
             (91511823, 18),
             (91511831, 19),
             (91533770, 20),
             (91533800, 21),
             (91539999, 22),
             (91551949, 23),
             (91555332, 24),
             (91560638, 25),
             (91562770, 26),
             (91562779, 27),
             (91606492, 28),
             (91613413, 29),
             (91613421, 30),
             (91614665, 31),
             (91622080, 32),
             (91710370, 33),
             (91862294, 

In [5]:
train_dataset = create_classification_dataset(ing2label,split='train',image_size=1024)
test_dataset = create_classification_dataset(ing2label,split='test',image_size=1024)

In [6]:
len(test_dataset), len(train_dataset)

(160, 215)

In [7]:
img,label = train_dataset[0]

In [8]:
ingredients_de = list(ing2name.values())
ingredients_en = [ "a " + translations_de_en[t] for t in ingredients_de]
ingredients_en

['a dressing_portion',
 'a sauerkraut',
 'a iceberg lettuce',
 'a bell pepper',
 'a potato pancakes',
 'a red cabbage',
 'a zucchini',
 'a mushrooms',
 'a lentils',
 'a apple sauce',
 'a cream',
 'a green beans',
 'a cauliflower',
 'a peas',
 'a carrots',
 'a rice',
 'a potato cubes',
 'a potatoes',
 'a brown sauce',
 'a light sauce',
 'a savoy cabbage',
 'a chicken',
 'a chicken strips',
 'a gravy',
 'a onion',
 'a bread dumplings',
 'a roast beef',
 'a roast pork neck',
 'a ham_mettwurst',
 'a grilled sausage',
 'a schnitzel',
 'a vegetable cream',
 'a pollock',
 'a egg noodles',
 'a mashed potatoes',
 'a meatballs',
 'a breaded fish fillet',
 'a lentil stew',
 'a tomato curry sauce',
 'a malt beer mustard sauce',
 'a coleslaw']

In [7]:
model = None # CLIPModel('PE-Core-T-16-384',ingredients_en)

In [10]:
probs = model(img.unsqueeze(0))
probs

tensor([[9.3391e-04, 1.4655e-03, 7.5307e-04, 3.4291e-01, 1.1454e-04, 4.1445e-05,
         2.4361e-03, 9.9012e-04, 1.5348e-03, 5.5049e-04, 3.3027e-02, 4.0798e-03,
         1.5790e-03, 7.0954e-04, 1.7672e-02, 2.2658e-03, 4.4298e-04, 2.7294e-03,
         1.2013e-01, 3.6962e-03, 1.0404e-04, 1.8082e-03, 6.5361e-05, 9.1589e-04,
         2.8521e-03, 6.5165e-03, 3.2778e-03, 1.3159e-03, 3.5009e-04, 4.7281e-04,
         1.4538e-07, 4.1156e-02, 4.9494e-04, 3.4038e-04, 1.9745e-02, 3.7362e-01,
         7.0815e-07, 5.9885e-03, 1.1112e-03, 1.2724e-03, 5.3097e-04]])

In [11]:
ingredients_en[label], ingredients_en[probs.argmax(dim=-1).item()]

('a chicken', 'a meatballs')

In [12]:
probs.argmax(dim=-1).item(), label

(35, 21)

In [8]:
vision_train_data = ClassificationVisionDataLoader.load_from_dataset(train_dataset,batch_size=8,shuffle=True,model=model)
vision_test_data = ClassificationVisionDataLoader.load_from_dataset(test_dataset,batch_size=8,shuffle=True,model=model)

In [9]:
config = DeepchecksConfig(train_test_validation=True,
                            data_integrity=True,
                            model_evaluation=False,
                            save_results=True,
                            save_display=False,
                            save_results_format='json',
                            parse_results=True,
                            output_dir='results')
                            
checks_runner = DeepchecksRunner(config)

In [10]:
results = checks_runner.run_suites(train_data=vision_train_data,test_data=vision_test_data)

/home/fadelco/.local/share/uv/python/cpython-3.11.13-linux-x86_64-gnu/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/fadelco/.local/share/uv/python/cpython-3.11.13-linux-x86_64-gnu/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/fadelco/.local/share/uv/python/cpython-3.11.13-linux-x86_64-gnu/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg

AttributeError: 'list' object has no attribute 'to_json'

In [19]:
results

{'train_test_validation': Train Test Validation Suite,
 'data_integrity': Data Integrity Suite,
 'model_evaluation': Model Evaluation Suite}

In [22]:
result = results['model_evaluation']

In [26]:
result

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_XAJI0Y6DPBHSAHXTHV3A3ZMF8">Model Evaluation S…

In [23]:
result.results 

[Prediction Drift: OrderedDict([('Samples Per Class', {'Drift score': np.float64(0.46978513481321343), 'Method': "Cramer's V"})]),
 Simple Model Comparison:             Model Metric  Class Class Name  Number of samples  Value
 0    Simple Model     F1      1          1                 40    0.0
 1    Simple Model     F1      4          4                 40    0.0
 2    Simple Model     F1     15         15                  0    0.0
 3    Simple Model     F1     34         34                 40    0.0
 4    Simple Model     F1     35         35                  0    0.0
 5    Simple Model     F1     37         37                 40    0.0
 6   Perfect Model     F1      1          1                 40    1.0
 7   Perfect Model     F1      4          4                 40    1.0
 8   Perfect Model     F1     34         34                 40    1.0
 9   Perfect Model     F1     37         37                 40    1.0
 10  Perfect Model     F1     15         15                  0    0.0
 11 

In [24]:
[type(r) for r in result.results]

[deepchecks.core.check_result.CheckResult,
 deepchecks.core.check_result.CheckResult,
 deepchecks.core.check_result.CheckResult,
 deepchecks.core.check_result.CheckResult,
 deepchecks.core.check_result.CheckResult,
 deepchecks.core.check_result.CheckFailure,
 deepchecks.core.check_result.CheckFailure,
 deepchecks.core.check_result.CheckFailure,
 deepchecks.core.check_result.CheckFailure,
 deepchecks.core.check_result.CheckFailure,
 deepchecks.core.check_result.CheckFailure]

In [30]:
json.loads(result.results[-1].to_json(with_display=False))

{'header': 'Label Property Outliers',
 'type': 'CheckFailure',
 'check': {'name': 'Label Property Outliers',
  'params': {'n_show_top': 3,
   'iqr_percentiles': [25, 75],
   'iqr_scale': 1.5,
   'min_samples': 10,
   'n_samples': 10000},
  'summary': 'Find outliers labels with respect to the given properties.'},
 'exception': 'task type classification does not have default label properties for label outliers.'}

In [25]:
[r.get_metadata().get('header') for r in result.results]

['Image Property Outliers',
 'Image Property Outliers',
 'Property Label Correlation',
 'Property Label Correlation',
 'Label Property Outliers',
 'Label Property Outliers']

In [57]:
[type(r)  for r in result.results[5].display]

[str,
 str,
 plotly.graph_objs._figure.Figure,
 str,
 str,
 str,
 plotly.graph_objs._figure.Figure]

In [58]:
[r for r in result.results[5].display if isinstance(r,str)]

['\n        <span>\n        The shown features are the image properties (brightness, aspect ratio, etc.) that are most important for the\n        domain classifier - the domain_classifier trained to distinguish between the train and test datasets.<br>\n        </span>\n        ',
 '\n    <span>\n    The percents of explained dataset difference are the importance values for the feature calculated\n    using `permutation_importance`.\n    </span><br><br>\n    ',
 '<h3>Main features contributing to drift</h3>',
 'Showing only the top 3 columns, you can change it using n_top_columns param',
 'For discrete distribution plots, showing the top 10 categories with largest difference between train and test.']

In [55]:
result.results[4] #.display[5]